In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import smartgd.model

In [3]:
best_xing_gen_param = smartgd.model.Generator.Params(
    num_blocks=10,
    block_depth=2,
    block_width=8,
    block_output_dim=8,
    edge_net_depth=1,
    edge_net_width=16,
    edge_attr_dim=2,
    node_attr_dim=2,
)

In [4]:
generator = smartgd.model.Generator(
    params=best_xing_gen_param,
)

In [8]:
discriminator = smartgd.model.Discriminator(
    params=...,
    edge_net_config=...,
    gnn_config=...,
    edge_feat_expansion=...,
    eps=...
)
discriminator_script = jit.script(discriminator)

In [10]:
sum(p.numel() for p in generator.parameters())

84200

In [11]:
sum(p.numel() for p in discriminator.parameters())

1047889

In [12]:
sum(p.numel() for p in generator.parameters()) + sum(p.numel() for p in discriminator.parameters())

1132089

In [5]:
dummy_discriminator = smartgd.model.DummyDiscriminator()
dummy_discriminator_script = jit.script(dummy_discriminator)

In [8]:
syncer.save(
    model=generator,
    name="smartgd_generator_best_xing",
    version="v1.0",
    configuration="default",
    latest=True,
    metadata=dict(
        description="SmartGD generator for best xing.",
        changelog=dict(
            major=[],
            minor=[],
            patch=[]
        )
    )
)

{'model_name': 'smartgd_generator_best_xing',
 'timestamp': '2023-04-14 15:35:18.029064',
 'class_name': 'Generator',
 'md5_digest': '3b174bf192b95db229ebdd54a030dc9c',
 'description': 'SmartGD generator for best xing.',
 'changelog': {'major': [], 'minor': [], 'patch': []}}

In [ ]:
syncer.save(
    model=generator,
    name="smartgd_generator",
    version="v0.4.2",
    configuration="default",
    latest=True,
    metadata=dict(
        description="SmartGD generator release candidate.",
        changelog=dict(
            major=[],
            minor=[
                "Decouple Generator from GraphLayout."
            ],
            patch=[
                "Revert"
            ]
        )
    )
)

In [9]:
syncer.save(
    model=discriminator,
    name="smartgd_discriminator",
    version="v0.4",
    configuration="default",
    latest=True,
    metadata=dict(
        description="SmartGD discriminator release candidate.",
        changelog=dict(
            major=[],
            minor=[
                "Decouple Discriminator from GraphLayout."
            ],
            patch=[]
        )
    )
)

{'model_name': 'smartgd_discriminator',
 'timestamp': '2023-03-24 09:14:43.003538',
 'class_name': 'Discriminator',
 'md5_digest': 'd55f329a2ea7e85f5adc2dcb28e4e2c8',
 'description': 'SmartGD discriminator release candidate.',
 'changelog': {'major': [],
  'minor': ['Decouple Discriminator from GraphLayout.'],
  'patch': []}}

In [7]:
syncer.save(
    model=dummy_discriminator,
    name="dummy_discriminator",
    version="v0.1",
    configuration="default",
    latest=True,
    metadata=dict(
        description="Discriminator wrapped around stress critic.",
        changelog=dict(
            major=[],
            minor=[],
            patch=[]
        )
    )
)

{'model_name': 'dummy_discriminator',
 'timestamp': '2023-03-10 14:14:25.433908',
 'class_name': 'DummyDiscriminator',
 'md5_digest': '1f229c7820a2af89325ebd017c6242ab',
 'description': 'Discriminator wrapped around stress critic.',
 'changelog': {'major': [], 'minor': [], 'patch': []}}

In [ ]:
assert False

# Debugging

In [4]:
from torch.jit import _drop
import torch

In [7]:
torch.__version__

'1.13.1'

In [6]:
torch.jit._drop

AttributeError: module 'torch.jit' has no attribute '_drop'

In [13]:
import torch_geometric as pyg
from tqdm.auto import tqdm

import smartgd.common.datasets
import smartgd.common.data

In [10]:
dataset = smartgd.common.datasets.RomeDataset()

In [11]:
loader = pyg.loader.DataLoader(dataset, batch_size=64, shuffle=False)

In [9]:
manager = smartgd.syncing.LayoutSyncer.get_default_syncer(dataset.name)
manager.clear_cache()

In [10]:
layout_store = manager.load(name="neato")

In [11]:
append_column = smartgd.datasets.BatchAppendColumn()

In [12]:
batch = next(iter(loader))
batch

DataBatch(edge_index=[2, 48892], pos=[1728, 2], G=[64], d_attr=[48892], full_index=[2, 48892], adj_index=[2, 4116], n=[64], m=[64], name=[64], batch=[1728], ptr=[65])

In [14]:
layout = smartgd.common.data.GraphLayout.from_data(batch)

In [16]:
discriminator(layout)

tensor([-20.2282, -24.9498, -14.4922, -19.1925, -32.7737, -42.2417, -21.6693,
        334.0409, -23.0981, -21.5613, -23.3014, -12.5310, -22.0713, -25.5861,
        -28.2184, -23.0297,  23.5783, -30.4462, -32.2476, -26.5339, -24.6811,
        -47.7293, -33.8589, -27.3999, -16.3149, -39.2427, -28.2900,   0.8720,
        -23.2233, -30.0251,  59.3673, -20.5245, -30.2681, -24.9897, 139.4994,
        -33.4278, -42.1947, -17.8935, -29.2227, -30.0344, -21.1950, -23.6042,
        -18.8434, -28.7404,  -3.9573, -29.4019, -33.5943, -46.1760, -19.5406,
        -19.9414, -15.7198, -27.7010, -29.6864, -17.6775, -31.4162, -24.7525,
        -17.0214, -88.1658, -20.0404, -23.9476, -38.8266, -16.8175, -25.1628,
        -23.9730], grad_fn=<ReshapeAliasBackward0>)

In [17]:
discriminator_script

RecursiveScriptModule(
  original_name=Discriminator
  (block): RecursiveScriptModule(
    original_name=DiscriminatorBlock
    (shared_edge_net): RecursiveScriptModule(
      original_name=DiscriminatorEdgeNet
      (edge_feature_provider): RecursiveScriptModule(
        original_name=EdgeFeatureExpansion
        (get_edge_feat): TracedModule(original_name=TracedModule)
      )
      (mlp): RecursiveScriptModule(
        original_name=MLP
        (linear_seq): RecursiveScriptModule(
          original_name=Sequential
          (0): RecursiveScriptModule(
            original_name=LinearLayer
            (dense): RecursiveScriptModule(original_name=Linear)
            (bn): RecursiveScriptModule(original_name=BatchNorm1d)
            (act): RecursiveScriptModule(original_name=LeakyReLU)
            (dp): RecursiveScriptModule(original_name=Dropout)
            (skip): RecursiveScriptModule(
              original_name=SkipConnection
              (proj): RecursiveScriptModule(original_

In [14]:
batch = append_column(batch=batch, tensor=torch.ones(1728, 2), name="new_entry")
batch

DataBatch(edge_index=[2, 48892], pos=[1728, 2], G=[64], d_attr=[48892], full_index=[2, 48892], adj_index=[2, 4116], n=[64], m=[64], name=[64], new_entry=[1728, 2], batch=[1728], ptr=[65])

In [16]:
[data.new_entry for data in batch.to_data_list()]

[tensor([[1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.]]),
 tensor([[1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [

In [30]:
data_list = batch.to_data_list()

In [40]:
for data in data_list:
    data.new_entry = torch.tensor([2])

In [41]:
batch = pyg.data.Batch.from_data_list(data_list)

In [43]:
len(batch)

64

In [42]:
batch.to_data_list()[0]["new_entry"]

tensor([2])

In [16]:
for batch in tqdm(loader):
    init_layout = smartgd.data.GraphLayout.from_data(batch)
    real_layout = smartgd.data.GraphLayout.from_data(batch, kvstore=layout_store)
    # fake_layout = generator(init_layout)
    try:
        # init_pred = discriminator(init_layout)
        real_pred = discriminator(real_layout)
        # fake_pred = discriminator(fake_layout)
    except Exception:
        print(batch.name)
        print(init_layout.pos.shape)
        print(real_layout.pos.shape)
        # print(fake_layout.pos.shape)
        break

  0%|          | 0/181 [00:00<?, ?it/s]

tensor([-3406.1494, -2557.2378, -3407.4443, -2074.8945, -1761.1205, -6028.1753,
        -1982.8461, -5653.0122, -1579.5651,  -880.7676, -1484.7103, -3406.8242,
        -3368.0547, -1820.3566, -2613.1536, -3295.4854, -6686.1240, -3012.8091,
        -2532.4937, -1607.4983, -1185.3990, -3687.8538, -3174.2295, -1518.8901,
        -1294.4954, -3661.2537, -2251.4443, -2556.8591, -2745.2258, -3669.4155,
        -3631.0818,  -906.8660, -2885.2773, -1617.7775, -3777.5596, -2523.0251,
        -4691.1655, -4439.7817, -3543.7527, -1927.4789, -1187.1056, -3747.4438,
         -632.1102, -1876.3882, -3382.0376, -1950.0465, -2063.1672, -3175.7441,
        -2802.1169, -2139.2515, -2089.0508, -1690.7231, -3328.0625,  -497.4807,
        -2090.4844, -2694.4092, -1900.6230, -5493.3721, -2569.6853, -3200.0425,
        -2209.3030, -2976.2461, -2286.5811, -2644.5830],
       grad_fn=<ReshapeAliasBackward0>)
['grafo113.28', 'grafo114.26', 'grafo115.30', 'grafo116.28', 'grafo117.20', 'grafo118.43', 'grafo119.24